## Reshaping and Pivot Tables

In [1]:
# URL: https://pandas.pydata.org/pandas-docs/stable/reshaping.html

In [2]:
import numpy as np
import pandas as pd

### Reshaping by pivoting DataFrame objects

In [3]:
"""
Data is often stored in CSV files or databases in so-called “stacked” or “record” format:
For the curious here is how the above DataFrame was created:
"""
import pandas.util.testing as tm; tm.N = 3
def unpivot(frame):
    N, K = frame.shape
    data = {'value': frame.values.ravel('F'),
            'variable': np.asarray(frame.columns).repeat(N),
            'date': np.tile(np.asarray(frame.index), K)}
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])
df = unpivot(tm.makeTimeDataFrame())
df

,date,variable,value
0,2000-01-03,A,-1.019632
1,2000-01-04,A,0.322897
2,2000-01-05,A,-0.077913
3,2000-01-03,B,0.329778
4,2000-01-04,B,-0.617621
5,2000-01-05,B,0.672014
6,2000-01-03,C,-0.528425
7,2000-01-04,C,-0.251722
8,2000-01-05,C,-0.679670
9,2000-01-03,D,1.380347


In [4]:
"""
To select out everything for variable C we could do:
"""
df[df['variable'] == 'C']

,date,variable,value
6,2000-01-03,C,-0.528425
7,2000-01-04,C,-0.251722
8,2000-01-05,C,-0.679670


In [5]:
"""
But suppose we wish to do time series operations with the variables. A better representation would be where 
the columns are the unique variables and an index of dates identifies individual observations. To reshape 
the data into this form, use the pivot function:
"""
df.pivot(index='date', columns='variable', values='value')

variable,A,B,C,D
date,,,,
2000-01-03,-1.019632,0.329778,-0.528425,1.380347
2000-01-04,0.322897,-0.617621,-0.251722,-1.175745
2000-01-05,-0.077913,0.672014,-0.679670,-0.884955


In [6]:
"""
If the values argument is omitted, and the input DataFrame has more than one column of values which are not 
used as column or index inputs to pivot, then the resulting “pivoted” DataFrame will have hierarchical columns 
whose topmost level indicates the respective value column:
"""
df['value2'] = df['value'] * 2
pivoted = df.pivot('date', 'variable')
pivoted

value                                  value2            \
variable           A         B         C         D         A         B   
date                                                                     
2000-01-03 -1.019632  0.329778 -0.528425  1.380347 -2.039263  0.659557   
2000-01-04  0.322897 -0.617621 -0.251722 -1.175745  0.645794 -1.235242   
2000-01-05 -0.077913  0.672014 -0.679670 -0.884955 -0.155827  1.344028   

                                
variable           C         D  
date                            
2000-01-03 -1.056850  2.760694  
2000-01-04 -0.503443 -2.351490  
2000-01-05 -1.359341 -1.769911

In [7]:
"""
You of course can then select subsets from the pivoted DataFrame:
"""
pivoted['value2']

variable,A,B,C,D
date,,,,
2000-01-03,-2.039263,0.659557,-1.056850,2.760694
2000-01-04,0.645794,-1.235242,-0.503443,-2.351490
2000-01-05,-0.155827,1.344028,-1.359341,-1.769911


### Reshaping by stackin and unstacking

In [8]:
"""
Closely related to the pivot function are the related stack and unstack functions currently available on 
Series and DataFrame. These functions are designed to work together with MultiIndex objects (see the section 
on hierarchical indexing). Here are essentially what these functions do:

    - stack: “pivot” a level of the (possibly hierarchical) column labels, returning a DataFrame with an 
    index with a new inner-most level of row labels.
    - unstack: inverse operation from stack: “pivot” a level of the (possibly hierarchical) row index to 7
    the column axis, producing a reshaped DataFrame with a new inner-most level of column labels.
    
The clearest way to explain is by example. Let’s take a prior example data set from the hierarchical indexing 
section:
"""
tuples = list(zip(*[['bar', 'bar', 'baz', 'baz',
                     'foo', 'foo', 'qux', 'qux'],
                    ['one', 'two', 'one', 'two',
                             'one', 'two', 'one', 'two']]))

index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
df = pd.DataFrame(np.random.randn(8, 2), index = index, columns = ['A', 'B'])
df

A         B
first second                    
bar   one     0.049391 -0.012290
      two     1.803184  1.930923
baz   one    -1.420972  0.479912
      two    -1.655144  0.496291
foo   one     1.522426 -0.545075
      two    -0.438870 -0.251000
qux   one    -0.464127 -0.504949
      two    -1.157245  1.289622

In [9]:
df2 = df[:4]
df2

A         B
first second                    
bar   one     0.049391 -0.012290
      two     1.803184  1.930923
baz   one    -1.420972  0.479912
      two    -1.655144  0.496291

In [13]:
"""
The stack function “compresses” a level in the DataFrame’s columns to produce either:

    - A Series, in the case of a simple column Index
    - A DataFrame, in the case of a MultiIndex in the columns

If the columns have a MultiIndex, you can choose which level to stack. The stacked level becomes the new 
lowest level in a MultiIndex on the columns:
"""
stacked = df2.stack()
stacked

first  second   
bar    one     A    0.049391
               B   -0.012290
       two     A    1.803184
               B    1.930923
baz    one     A   -1.420972
               B    0.479912
       two     A   -1.655144
               B    0.496291
dtype: float64

In [14]:
"""
With a “stacked” DataFrame or Series (having a MultiIndex as the index), the inverse operation of stack 
is unstack, which by default unstacks the last level:
"""
stacked.unstack()

A         B
first second                    
bar   one     0.049391 -0.012290
      two     1.803184  1.930923
baz   one    -1.420972  0.479912
      two    -1.655144  0.496291

In [15]:
stacked.unstack(1)

second        one       two
first                      
bar   A  0.049391  1.803184
      B -0.012290  1.930923
baz   A -1.420972 -1.655144
      B  0.479912  0.496291

In [16]:
stacked.unstack(0)

first          bar       baz
second                      
one    A  0.049391 -1.420972
       B -0.012290  0.479912
two    A  1.803184 -1.655144
       B  1.930923  0.496291

In [17]:
"""
If the indexes have names, you can use the level names instead of specifying the level numbers:
"""
stacked.unstack('second')

second        one       two
first                      
bar   A  0.049391  1.803184
      B -0.012290  1.930923
baz   A -1.420972 -1.655144
      B  0.479912  0.496291

In [18]:
"""
Notice that the stack and unstack methods implicitly sort the index levels involved. Hence a call to 
stack and then unstack, or vice versa, will result in a sorted copy of the original DataFrame or Series:
"""
index = pd.MultiIndex.from_product([[2, 1], ['a', 'b']])
df = pd.DataFrame(np.random.randn(4), index = index, columns = ['A'])
df

A
2 a -0.201512
  b  1.802406
1 a -0.747744
  b -0.336034

In [19]:
all(df.unstack().stack() == df.sort_index())
# while the above code will raise a TypeError if the call to sort_index is removed.

True

### Multiple Levels

In [20]:
"""
You may also stack or unstack more than one level at a time by passing a list of levels, 
in which case the end result is as if each level in the list were processed individually.
"""
columns = pd.MultiIndex.from_tuples([
    ('A', 'cat', 'long'), ('B', 'cat', 'long'),
    ('A', 'dog', 'short'), ('B', 'dog', 'short')],
    names=['exp', 'animal', 'hair_length']
)
df = pd.DataFrame(np.random.randn(4, 4), columns = columns)
df

exp,A,B,A,B
animal,cat,cat,dog,dog
hair_length,long,long,short,short
0,-0.794064,-0.966008,0.486712,-0.898411
1,0.672372,0.029072,0.352156,-2.365901
2,1.151656,-0.350271,1.306237,-0.632064
3,0.483692,0.966558,-0.667056,0.146307


In [21]:
df.stack( level = ['animal', 'hair_length'])

exp                          A         B
  animal hair_length                    
0 cat    long        -0.794064 -0.966008
  dog    short        0.486712 -0.898411
1 cat    long         0.672372  0.029072
  dog    short        0.352156 -2.365901
2 cat    long         1.151656 -0.350271
  dog    short        1.306237 -0.632064
3 cat    long         0.483692  0.966558
  dog    short       -0.667056  0.146307

In [24]:
"""
The list of levels can contain either level names or level numbers (but not a mixture of the two).
"""
df.stack(level = [1, 2])

exp                          A         B
  animal hair_length                    
0 cat    long        -0.794064 -0.966008
  dog    short        0.486712 -0.898411
1 cat    long         0.672372  0.029072
  dog    short        0.352156 -2.365901
2 cat    long         1.151656 -0.350271
  dog    short        1.306237 -0.632064
3 cat    long         0.483692  0.966558
  dog    short       -0.667056  0.146307

### Missing Data

In [29]:
"""
These functions are intelligent about handling missing data and do not expect each subgroup within the 
hierarchical index to have the same set of labels. They also can handle the index being unsorted 
(but you can make it sorted by calling sort_index, of course). Here is a more complex example:
"""
columns = pd.MultiIndex.from_tuples([('A', 'cat'), ('B', 'dog'),
                                     ('B', 'cat'), ('A', 'dog')],
                                    names=['exp', 'animal'])
index = pd.MultiIndex.from_product([('bar', 'baz', 'foo', 'qux'),
                                    ('one', 'two')],
                                   names=['first', 'second'])
df = pd.DataFrame(np.random.rand(8, 4), index = index, columns = columns)
df

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     0.019485  0.749868  0.711823  0.402694
      two     0.017098  0.232342  0.226083  0.894382
baz   one     0.537249  0.238105  0.765866  0.947117
      two     0.999747  0.811028  0.140034  0.086420
foo   one     0.870400  0.900491  0.501919  0.141529
      two     0.426912  0.235210  0.892711  0.548489
qux   one     0.901192  0.591345  0.565448  0.147771
      two     0.534838  0.184768  0.224924  0.176892

In [31]:
df2 = df.iloc[[0, 1, 2, 3, 4, 5, 7]]
df2

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     0.019485  0.749868  0.711823  0.402694
      two     0.017098  0.232342  0.226083  0.894382
baz   one     0.537249  0.238105  0.765866  0.947117
      two     0.999747  0.811028  0.140034  0.086420
foo   one     0.870400  0.900491  0.501919  0.141529
      two     0.426912  0.235210  0.892711  0.548489
qux   two     0.534838  0.184768  0.224924  0.176892